In [1]:
import keras
import numpy as np
from neuralStellar import *
import pymc3 as pm
import theano as T
import theano.tensor as TT

Using TensorFlow backend.


In [2]:
m1=NNmodel('evo',['mass','age','feh','MLT'],['L','Teff','delnu'])
m1.loadModel('Ran_files/small_grid_28/small_grid_model.h5')

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense (Dense)                (None, 128)               640       
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_3 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_4 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_5 (Dense)              (None, 128)               16512 

In [3]:
rawdata=[]
for line in open('M67_data.txt','r').read().split('\n'):
    rawdata.append(line.split(' '))
print(rawdata[0])
data=[]
for stardata in rawdata[1:]:
    if len(stardata)==len(rawdata[0]):
        data.append(stardata)

def readTable(index, data):
    obss=[]
    errs=[]
    for stardata in data:
        obs=float(stardata[index].split('(')[0])
        err=stardata[index].split('(')[1][:-1]
        if '.' not in err:
            err=float('0.'+err)
        else: err=float(err)
        obss.append(obs)
        errs.append(err)
    return np.array(obss), np.array(errs)
        
ages, age_sigmas =  readTable(-1,data)
delnus, delnu_errs = readTable(4,data)
Teffs, Teff_errs = readTable(5,data)
    
Xobs=ages
sigma_obs=np.average(age_sigmas)
N=len(ages)
print(np.mean(ages),sigma_obs)

['EPIC_ID', 'WOCS_ID', 'Class', 'Î½max', 'âˆ†Î½', 'Teff/K', 'Rsc', 'Msc', 'log_gsc', 'Rcorr', 'Mcorr', 'Rgrid', 'Mgrid', 'log_ggrid', 'Agegrid']
4.787096774193548 1.5612903225806454


In [42]:
m1.getWeights()

weights=m1.model.get_weights()
def manualPredict(inputs):
    no_hidden_layers = len(weights)/2-1
    input shape = 2D array with N rows of parameters, each of M columns of stars
    xx=inputs
    for i in np.arange(0,no_hidden_layers)*2:
        i=int(i)
        xx=TT.nnet.relu(pm.math.dot(weights[i].T,xx).T+weights[i+1]).T
    xx=(TT.dot(weights[-2].T,xx).T+weights[-1]).T
    return xx

#mass mean - lognormal
M_mu_mu = 1
M_mu_sigma = 1
#mass sigma - half normal
M_sigma_sigma = 5

#age mean - lognormal
Age_mu_mu = 4.5
Age_mu_sigma = 1
#age sigma - half normal
Age_sigma_sigma = 5

#feh mean - normal
feh_mu_mu = 0
feh_mu_sigma = 1
#feh sigma - half normal
feh_sigma_sigma = 1

#alpha MLT mean - normal
MLT_mu_mu = 2
MLT_mu_sigma = 1
#alpha MLT sigma - half normal
MLT_sigma_sigma = 0.5

N = len(data)

model = pm.Model()
with model:
    M_mu = pm.Lognormal('mean_mass', np.log10(M_mu_mu), M_mu_sigma)
    M_sigma = pm.HalfNormal('spread_mass', M_sigma_sigma)
    
    Age_mu = pm.Lognormal('mean_age', np.log10(Age_mu_mu), Age_mu_sigma)
    Age_sigma = pm.HalfNormal('spread_age', Age_sigma_sigma)
    
    #feh_mu = pm.Normal('mean_feh', feh_mu_mu, feh_mu_sigma)
    #feh_sigma = pm.HalfNormal('spread_feh', feh_sigma_sigma)
    
    #MLT_mu = pm.Normal('mean_MLT', MLT_mu_mu, MLT_mu_sigma)
    #MLT_sigma = pm.HalfNormal('spread_MLT', MLT_sigma_sigma)
    
    M = pm.Normal('mass', M_mu, M_sigma, shape=N)
    log_m = np.log10(M)
    Age = pm.Normal('age', Age_mu, Age_sigma, shape=N)
    log_a = np.log10(Age)
    #feh = pm.Normal('feh', feh_mu, feh_sigma, shape=N)
    feh = np.zeros(N)
    #MLT = pm.Normal('MLT', MLT_mu, MLT_sigma, shape=N)
    MLT = np.ones(N)*1.9
    
    obs = pm.Deterministic('obs',manualPredict([log_m, log_a, feh, MLT]))
    
    #obs_L = pm.Normal('obs_L',10**obs[0],0.1, observed=1.0)
    obs_Teff = pm.Normal('obs_Teff',10**obs[1],Teff_errs, observed=Teffs)
    obs_delnu = pm.Normal('obs_delnu',10**obs[2],delnu_errs, observed=delnus)

#start = {'mass': 0.8, 'age': 2.5}
with model:
    trace = pm.sample()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [age, mass, spread_age, mean_age, spread_mass, mean_mass]
Sampling 2 chains:  28%|███████████████▉                                         | 560/2000 [00:08<00:22, 63.94draws/s]


RuntimeError: Chain 0 failed.

In [43]:
a=np.array([[1,2,3,4],[10,20,30,40],[100,200,300,400]])
b=np.array([[1,1.5],[2,2.5],[3,3.5],[4,4.5]])
#c=np.array([[0.1,0.2],[0.1,0.2],[0.1,0.2]]).T
#c=np.repeat([[0.1,0.2]],len(a),axis=0).T
c=np.array([0.1,0.2])
print(c)
#a=T.shared(a)
#b=T.shared(b)
#print(a.shape.eval(),b.shape.eval())
print((TT.dot(b.T,a.T).T+c).T.eval())

[0.1 0.2]
[[  30.1  300.1 3000.1]
 [  35.2  350.2 3500.2]]


In [22]:
print(a.shape.eval()[0])

3


In [51]:
k=T.shared(weights[0])
TT.dot(k,k.T)

dot.0

In [20]:
TT.reshape(a,newshape=(3,1,4)).eval()

array([[[  1,   2,   3,   4]],

       [[ 10,  20,  30,  40]],

       [[100, 200, 300, 400]]])